In [12]:
import torch
import os
from pandas import read_csv
from torch.optim import SGD
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from config import DATA_DIR, MODELS_DIR
from src.loss import Loss
from src.model import HistogramMF
from src.runner import Runner
from src.create_dataset import create_dataset
from src.data_processor import DataProcessor
from src.data_encoder import DataEncoder

DF_PATH = (
    f"{DATA_DIR}"
    f"/DEAM/annotations/annotations per each rater/"
    f"song_level/static_annotations_songs_1_2000_raw.csv"
)

In [13]:
columns = ["workerID", "SongId", "Valence", "Arousal"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "Valence", "Arousal"]
original_df.head()

,user_id,item_id,Valence,Arousal
0,6010bbc8e7ef4b21fa38f9c3a9754ef3,2,5,2
1,3c888e77b992ae3cd2adfe16774e23b9,2,2,3
2,2afd218c3aecb6828d2be327f8b9c46f,2,3,3
3,fd5b08ce362d855ca9152a894348130c,2,4,4
4,9c8073214a052e414811b76012df8847,2,2,2


In [14]:
valence_dataframe = original_df[["user_id", "item_id", "Valence"]].copy()
valence_dataframe.columns = ["user_id", "item_id", "rating"]

data_encoder = DataEncoder(original_df=valence_dataframe)
data_processor = DataProcessor(original_df=valence_dataframe)

n_users = valence_dataframe.user_id.nunique()
n_items = valence_dataframe.item_id.nunique()

min_rating = min(valence_dataframe.rating.values)
max_rating = max(valence_dataframe.rating.values)

valence_model = HistogramMF(
    n_users=n_users,
    n_items=n_items,
    data_encoder=data_encoder,
    data_processor=data_processor,
    min_rating=min_rating,
    max_rating=max_rating,
)

if os.path.exists(f"{MODELS_DIR}/DEAM/valence.pt"):
    valence_model.load_state_dict(torch.load(f"{MODELS_DIR}/DEAM/valence.pt"))
else:
    epochs = 100

    criterion = Loss()
    optimizer = SGD(valence_model.parameters(), lr=5, weight_decay=1e-7)
    runner = Runner(model=valence_model, criterion=criterion, optimizer=optimizer)

    train_set = create_dataset(data_encoder=data_encoder)
    train_load = DataLoader(train_set, batch_size=1000, shuffle=True)

    with SummaryWriter(f"runs/DEAM/dev/valence") as writer:
        for epoch in range(epochs):
            epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
            print(f"epoch={epoch + 1}, loss={epoch_loss}")

    torch.save(valence_model.state_dict(), f"{MODELS_DIR}/DEAM/valence.pt")

  0%|          | 0/17464 [00:00<?, ?it/s]

  0%|          | 0/17464 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:02<00:00,  6.07batch/s, train_loss=0.0732]


epoch=1, loss=1.4918756646123428


100%|██████████| 18/18 [00:02<00:00,  8.25batch/s, train_loss=0.0632]


...

epoch=98, loss=0.01783483492608728


100%|██████████| 18/18 [00:02<00:00,  6.90batch/s, train_loss=0.00111] 


epoch=99, loss=0.017989741171228476


100%|██████████| 18/18 [00:02<00:00,  8.06batch/s, train_loss=0.00123] 

epoch=100, loss=0.017744535649644917


In [15]:
arousal_dataframe = original_df[["user_id", "item_id", "Arousal"]].copy()
arousal_dataframe.columns = ["user_id", "item_id", "rating"]

data_encoder = DataEncoder(original_df=arousal_dataframe)
data_processor = DataProcessor(original_df=arousal_dataframe)

n_users = arousal_dataframe.user_id.nunique()
n_items = arousal_dataframe.item_id.nunique()

min_rating = min(arousal_dataframe.rating.values)
max_rating = max(arousal_dataframe.rating.values)

arousal_model = HistogramMF(
    n_users=n_users,
    n_items=n_items,
    data_encoder=data_encoder,
    data_processor=data_processor,
    min_rating=min_rating,
    max_rating=max_rating,
)

if os.path.exists(f"{MODELS_DIR}/DEAM/arousal.pt"):
    valence_model.load_state_dict(torch.load(f"{MODELS_DIR}/DEAM/arousal.pt"))
else:
    epochs = 100

    criterion = Loss()
    optimizer = SGD(arousal_model.parameters(), lr=5, weight_decay=1e-7)
    runner = Runner(model=arousal_model, criterion=criterion, optimizer=optimizer)

    train_set = create_dataset(data_encoder=data_encoder)
    train_load = DataLoader(train_set, batch_size=1000, shuffle=True)

    with SummaryWriter(f"runs/DEAM/dev/arousal") as writer:
        for epoch in range(epochs):
            epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
            print(f"epoch={epoch + 1}, loss={epoch_loss}")

    torch.save(arousal_model.state_dict(), f"{MODELS_DIR}/DEAM/arousal.pt")

  0%|          | 0/17464 [00:00<?, ?it/s]

  0%|          | 0/17464 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:04<00:00,  4.12batch/s, train_loss=0.0801]


epoch=1, loss=1.5037995927744898


100%|██████████| 18/18 [00:03<00:00,  5.46batch/s, train_loss=0.0668]


...

epoch=98, loss=0.015199870771889027


100%|██████████| 18/18 [00:02<00:00,  6.88batch/s, train_loss=0.000791]


epoch=99, loss=0.015528864988479121


100%|██████████| 18/18 [00:02<00:00,  6.49batch/s, train_loss=0.00076] 

epoch=100, loss=0.015288945635331088


In [16]:
from pandas import DataFrame
from collections import namedtuple

Row = namedtuple("Row", "user_id item_id Valence Arousal")

data_encoder = DataEncoder(original_df=original_df)
dataframe_after_mf = []

with torch.no_grad():
    for (index, user_id, item_id, valence, arousal) in original_df.itertuples():
        encoded_user_id = data_encoder.get_encoded_user_id(original_id=user_id)
        encoded_item_id = data_encoder.get_encoded_item_id(original_id=item_id)

        user_id_as_tensor = torch.LongTensor([encoded_user_id])
        item_id_as_tensor = torch.LongTensor([encoded_item_id])
        valence_output = valence_model(users=user_id_as_tensor, items=item_id_as_tensor,).squeeze()[0]
        arousal_output = arousal_model(users=user_id_as_tensor, items=item_id_as_tensor,).squeeze()[0]
        predicted_valence = torch.round(valence_output).item()
        predicted_arousal = torch.round(arousal_output).item()

        dataframe_after_mf.append(Row(user_id=user_id, item_id=item_id, Valence=predicted_valence, Arousal=predicted_arousal))

df_after_mf = DataFrame(dataframe_after_mf, columns=["user_id", "item_id", "Valence", "Arousal"])
df_after_mf.head()

,user_id,item_id,Valence,Arousal
0,6010bbc8e7ef4b21fa38f9c3a9754ef3,2,5.0,2.0
1,3c888e77b992ae3cd2adfe16774e23b9,2,2.0,3.0
2,2afd218c3aecb6828d2be327f8b9c46f,2,3.0,3.0
3,fd5b08ce362d855ca9152a894348130c,2,4.0,4.0
4,9c8073214a052e414811b76012df8847,2,2.0,2.0


In [19]:
mask = ((original_df["Valence"] == df_after_mf["Valence"]) & (original_df["Arousal"] == df_after_mf["Arousal"]))
changes = original_df[mask].copy()
changes["New Valence"] = df_after_mf.Valence
changes["New Arousal"] = df_after_mf.Arousal
print(f"Number of hits: {len(changes)} / {len(original_df)}")
changes.head(len(changes))

Number of hits: 17341 / 17464


,user_id,item_id,Valence,Arousal,New Valence,New Arousal
0,6010bbc8e7ef4b21fa38f9c3a9754ef3,2,5,2,5.0,2.0
1,3c888e77b992ae3cd2adfe16774e23b9,2,2,3,2.0,3.0
2,2afd218c3aecb6828d2be327f8b9c46f,2,3,3,3.0,3.0
3,fd5b08ce362d855ca9152a894348130c,2,4,4,4.0,4.0
4,9c8073214a052e414811b76012df8847,2,2,2,2.0,2.0
...,...,...,...,...,...,...
17459,607f6e34a0b5923333f6b16d3a59cc98,2000,6,5,6.0,5.0
17460,78b5e9744073532cc376976b5fc6b2fc,2000,7,7,7.0,7.0
17461,7cecbffe1da5ae974952db6c13695afe,2000,4,5,4.0,5.0
17462,ed7ed76453bd846859f5e6b9149df276,2000,6,7,6.0,7.0
